In [2]:
import pandas as pd
import numpy as np

In [3]:
costumers = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_customers_dataset.csv')
itens = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_order_items_dataset.csv')
payments = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_order_payments_dataset.csv')
reviews = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_order_reviews_dataset.csv')
orders = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_orders_dataset.csv')
products = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_products_dataset.csv')
sellers = pd.read_csv('https://raw.githubusercontent.com/edunb01/dotfiles/master/olist_sellers_dataset.csv')

In [4]:
df = itens.merge(payments)
df = df.merge(sellers)
df = df.merge(products)
df = df.merge(reviews)
df = df.merge(orders)
df = df.merge(costumers)
df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,...,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.90,17.96,1,boleto,1,...,delivered,2017-06-28 11:52:20,2017-06-29 02:44:11,2017-07-05 12:00:33,2017-07-13 20:39:29,2017-07-26 00:00:00,0fb8e3eab2d3e79d92bb3fffbb97f188,75800,jatai,GO
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.90,18.33,1,credit_card,2,...,delivered,2018-05-18 10:25:53,2018-05-18 12:31:43,2018-05-23 14:05:00,2018-06-04 18:34:26,2018-06-07 00:00:00,3419052c8c6b45daf79c1e426f9e9bcb,30720,belo horizonte,MG
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.90,16.17,1,credit_card,3,...,delivered,2017-08-01 18:38:42,2017-08-01 18:55:08,2017-08-02 19:07:36,2017-08-09 21:26:33,2017-08-25 00:00:00,e7c828d22c0682c1565252deefbe334d,83070,sao jose dos pinhais,PR
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.90,13.29,1,credit_card,4,...,delivered,2017-08-10 21:48:40,2017-08-10 22:05:11,2017-08-11 19:43:07,2017-08-24 20:04:21,2017-09-01 00:00:00,0bb98ba72dcc08e95f9d8cc434e9a2cc,36400,conselheiro lafaiete,MG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117324,fd607d0a02cb276bc764a15f80f0423f,2,02ad78835ec3f72e750d1dfa6a7e11b3,2a73cba571d90c694b7caca072ccf6ce,2018-08-02 02:24:15,171.39,19.35,1,boleto,1,...,delivered,2018-07-26 09:13:04,2018-07-27 02:24:15,2018-08-07 09:28:00,2018-08-16 17:12:37,2018-08-16 00:00:00,c13401002a5f9b60befb4c720de50be0,41820,salvador,BA
117325,fdaf98feac227eb978d8f33f36e0231d,1,f739e9151702508b18f796c53005e5e9,d1aa1ec0839dcab73a6161130eb1f94a,2017-04-27 05:10:28,119.90,16.14,1,boleto,1,...,delivered,2017-04-19 10:20:51,2017-04-21 05:10:28,2017-05-04 15:06:24,2017-05-24 10:52:43,2017-05-22 00:00:00,3d50a44231c2a153219cef76ee3e445d,22793,rio de janeiro,RJ
117326,fe68b52db13993f58175fa589125d345,1,f4135cbdece8245560f7be179533797a,3fefda3299e6dfaea3466ef346a3571a,2017-08-17 02:55:27,209.00,16.21,1,boleto,1,...,delivered,2017-08-11 10:15:22,2017-08-12 02:55:27,2017-08-14 22:03:19,2017-08-17 17:47:55,2017-09-04 00:00:00,52a953541861bbb4b79b39df24e00243,13485,limeira,SP
117327,ff701a7c869ad21de22a6994237c8a00,1,5ff4076c0f01eeba4f728c9e3fa2653c,3e35a8bb43569389d3cebef0ce820f69,2018-04-18 20:10:33,27.90,14.44,1,credit_card,1,...,delivered,2018-04-12 19:47:40,2018-04-12 20:10:33,2018-04-18 00:41:29,2018-04-20 17:48:39,2018-04-30 00:00:00,ddb2249e0316d365ceae561c0f011bce,12280,cacapava,SP


In [ ]:
reviews['review_score'].value_counts()

In [8]:
reviews['review_score2'] = reviews['review_score'].apply(lambda x: '1' if x > 4 else '0')

In [10]:
reviews['review_score2'].value_counts()

review_score2
1    57328
0    41896
Name: count, dtype: int64

In [13]:
import numpy as np
import pandas as pd
import nltk
import unidecode 
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier, Pool
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [26]:
dataset = reviews[['review_comment_message','review_score2']].rename(columns={'review_comment_message':'texto','review_score2':'rótulo'})
dataset

,texto,rótulo
0,NaN,0
1,NaN,1
2,NaN,1
3,Recebi bem antes do prazo estipulado.,1
4,Parabéns lojas lannister adorei comprar pela I...,1
...,...,...
99219,NaN,1
99220,NaN,1
99221,"Excelente mochila, entrega super rápida. Super...",1
99222,NaN,0


In [27]:
def NLP(dataset): # Input: Dataset inteiro: 2 colunas: texto + rótulo ; qual algoritmo usar

    # Bloco 1: Processamento do texto
    corpus = []
    for i in range(0,dataset.shape[0]):
        texto = re.sub('[^a-zA-Z]',' ',unidecode.unidecode(dataset['texto'][i]) )
        texto = texto.lower()
        texto = texto.split()
        ps = PorterStemmer()
        texto = [ps.stem(word) for word in texto if not word in set(stopwords.words('portuguese'))]
        texto = ' '.join(texto)
        corpus.append(texto)

    # Bloco 2: Embedding do texto
    # BoW
    cv=CountVectorizer(max_features = 5000)
    X=cv.fit_transform(corpus).toarray()
    y=dataset.iloc[:,1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

    # Bloco 3 : Algorítmos de NLP
    metricas = {}

    model_RF = RandomForestClassifier(n_estimators = 400,criterion = "entropy",random_state = 0)
    model_RF.fit(X_train,y_train)     
    model_XGB = XGBClassifier(n_estimators = 400, max_depth = 2, learning_rate = 1, objective = 'binary:logistic')
    model_XGB.fit(X_train, y_train)
    model_GBC = GradientBoostingClassifier()
    model_GBC.fit(X_train, y_train)
    model_SVM = svm.SVC()
    model_SVM.fit(X_train, y_train)
    model_GNB = GaussianNB()
    model_GNB.fit(X_train, y_train)
    model_MNB = MultinomialNB()
    model_MNB.fit(X_train, y_train)
    model_CNB = ComplementNB()
    model_CNB.fit(X_train, y_train)

    # Bloco 4: Acumulador/organizador de resultados
    metricas = {}

    # Random Forest:
    y_pred_RF = model_RF.predict(X_test)
    metricas["Random Forest"] = {
        "acurácia": accuracy_score(y_test, y_pred_RF),
        "precisão": precision_score(y_test, y_pred_RF),
        "f1 score": f1_score(y_test, y_pred_RF)
    }

    # XGBoost:
    y_pred_XGB = model_XGB.predict(X_test)
    metricas["XGBoost"] = {
        "acurácia": accuracy_score(y_test, y_pred_XGB),
        "precisão": precision_score(y_test, y_pred_XGB),
        "f1 score": f1_score(y_test, y_pred_XGB)
    }

    y_pred_gbc = model_GBC.predict(X_test)
    metricas["Gradient Boosting Classifier"] = {
        "acurácia": accuracy_score(y_test, y_pred_gbc),
        "precisão": precision_score(y_test, y_pred_gbc),
        "f1 score": f1_score(y_test, y_pred_gbc)
    }

    # SVM:
    y_pred_svm = model_SVM.predict(X_test)
    metricas["SVM"] = {
        "acurácia": accuracy_score(y_test, y_pred_svm),
        "precisão": precision_score(y_test, y_pred_svm),
        "f1 score": f1_score(y_test, y_pred_svm)
    }

    # Naive Bayes Gaussian:
    y_pred_GNB = model_GNB.predict(X_test)
    metricas["Naive Bayes Gaussian"] = {
        "acurácia": accuracy_score(y_test, y_pred_GNB),
        "precisão": precision_score(y_test, y_pred_GNB),
        "f1 score": f1_score(y_test, y_pred_GNB)
    }

    # Naive Bayes Multinomial:
    y_pred_MNB = model_MNB.predict(X_test)
    metricas["Naive Bayes Multinomial"] = {
        "acurácia": accuracy_score(y_test, y_pred_MNB),
        "precisão": precision_score(y_test, y_pred_MNB),
        "f1 score": f1_score(y_test, y_pred_MNB)
    }

    # Naive Bayes Complement:
    y_pred_CNB = model_CNB.predict(X_test)
    metricas["Naive Bayes Complement"] = {
        "acurácia": accuracy_score(y_test, y_pred_CNB),
        "precisão": precision_score(y_test, y_pred_CNB),
        "f1 score": f1_score(y_test, y_pred_CNB)
    }

    # Bloco 5: Saída
    # Crie o DataFrame a partir do dicionário de métricas
    df = pd.DataFrame(metricas)

    # Transponha o DataFrame para que os nomes dos algoritmos sejam nas linhas e as métricas nas colunas
    df = df.transpose()

    # Imprima o DataFrame como uma tabela
    print(df)

In [32]:
dataset['texto'][4]

'Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa'

In [39]:
str(dataset['texto']).encode('ASCII')
Texto = re.sub('[^a-zA-Z]',' ',unidecode.unidecode(dataset['texto'][1]) )

UnicodeEncodeError: 'ascii' codec can't encode character '\xe9' in position 250: ordinal not in range(128)

In [33]:
    # Bloco 1: Processamento do texto
    corpus = []
    for i in range(0,dataset.shape[0]):
        texto = re.sub('[^a-zA-Z]',' ',unidecode.unidecode(dataset['texto'][i]) )
        texto = texto.lower()
        texto = texto.split()
        ps = PorterStemmer()
        texto = [ps.stem(word) for word in texto if not word in set(stopwords.words('portuguese'))]
        texto = ' '.join(texto)
        corpus.append(texto)

AttributeError: 'float' object has no attribute 'encode'

In [25]:
NLP(dataset)

AttributeError: 'float' object has no attribute 'encode'